## 합성곱 연산
### x와 w가 있을 때 w를 뒤집은 w_r을 x의 왼쪽 끝자리에 맞춘 뒤 
### 각 배열 원소끼리 곱한 뒤 합한 값을 구하며 이를 오른쪽으로 한칸 씩 옮겨가며 값들을 구함. 
### 이를 기호로 나타내면 x*w이며 곱하기와 다른 뜻이므로 혼동하지 말 것.

In [2]:
# 합성곱 구현
import numpy as np
w = np.array([2,1,5,3])
x = np.array([2,8,3,7,1,2,0,4,5])

In [3]:
w_r = np.flip(w) # 배열 뒤집기
print(w)
print(w_r)

[2 1 5 3]
[3 5 1 2]


In [4]:
# 이렇게도 뒤집기 가능
w_r = w[::-1]
print(w_r)

[3 5 1 2]


In [6]:
# 넘파이의 점 곱(dot 연산)으로 합성곱 수행 
for i in range(6):
    print(np.dot(x[i:i+4], w_r)) # 왼쪽부터 차례대로 w배열의 크기에 해당하는 x의 부분 배열과 w를 내적한다. 

63
48
49
28
21
20


In [9]:
# 싸이파이로 합성곱 수행(싸이파이가 합성곱을 위한 함수 convolve() 제공)
from scipy.signal import convolve
convolve(x, w, mode='valid') # 합성곱의 결과들을 담은 배열 반환

array([63, 48, 49, 28, 21, 20])

## 합성곱 신경망은 실은 합성곱을 사용하지 않는다!
### 그럼 왜 이름을 그렇게 지은거야? 
### 합성곱 신경망은 합성곱 연산 대신 교차 상관 연산을 사용한다. 교차 상관과 합성곱의 차이는 합성곱은 w를 뒤집은 w_r을 쓰고 교차 상관은 w를 뒤집지 않고 w를 곱셈에 쓴다는 것이다.

In [12]:
# 교차상관은 correlate()를 사용한다.
from scipy.signal import correlate
correlate(x, w, mode='valid') 

array([48, 57, 24, 25, 16, 39])

## 합성곱 신경망에서 합성곱 대신 교차 상관을 사용하는 이유?
### 답은 가중치 배열을 초기화하는 과정에서 알 수 있다. w는 이름에서 알 수 있듯 가중치 배열을 의미하는데 가중치 배열은 무작위로 초기화되기 때문에 뒤집든 말든 아무런 상관이 없다. 
### 그럼 걍 이름에 맞게 뒤집어서 쓰지 왜 뒤집지 않고 교차 상관 연산을 사용하는거야 햇갈리게? 라는 의문이 생길 수 있다. 난 생겼다. 
### 그 이유는 '관례상' 합성곱 신경망에 교차 상관 연산을 사용했기 때문에 그냥 교차 상관 연산을 사용하는거다. 허허...뭔 이런 관례가 있나 싶은데 뭐, 이렇게 쓴다고 하니 햇갈리지 말게 잘써야지. 

## 패딩과 스트라이드
### 패딩(padding) : 원본 배열(미끄러지는 배열, 이 경우에는 x)의 양 끝에 빈 원소를 추가하는 것을 말한다. 
### 스트라이드(stride) : 미끄러지는 배열의 간격을 조절하는 것을 말한다. 
### 이 두 개념이 어떻게 쓰이냐에 따라 밸리드 패딩, 풀 패딩, 세임 패딩으로 나뉜다.

### correlate()에서 mode = 'valid'라고 했다. 이는 벨리드 패딩(valid padding)을 적용한 것이다. 
### 밸리드 패딩은 원본 배열에 패딩을 추가하지 않고 배열이 원본 배열의 끝으로 갈 때 까지 교차 상관 연산을 수행한다. -> 밸리드 패딩으로 얻는 배열의 크기는 원본 배열보다 항상 작다. 
### 밸리드 패딩의 특징 중 하나는 인덱스가 무엇이냐에 따라 각 원소들이 연산에 참여하는 정도가 다르다는 것이다. 앞서 예제 코드에서 알 수 있듯 맨 처음에 나온 값은 한 번만 연산하고 중간에 있는 값들은 4번이나 연산했다. 
### 이는 곳 처음, 끝의 원소들이 연산에 참여한는 횟수가 적다는 뜻으로도 말할 수 있다. 

### 이를 해결하기 위해 원본 배열의 양 끝에 가상의 원소(0이 들어있는 원소)를 추가한다. 이를 제로 패딩(zero padding)이라고 한다. 적절한 수의 제로 패딩을 추가할 경우 모든 원소들이 연산에 동일하게 참여할 수 있게 된다. 
### 만약 배열의 모든 원소가 동일하게 연산에 참여하는 방식을 풀 패딩(full padding)이라고 한다. 

In [13]:
# 풀 패딩 구현 -> correlate()에서 매개변수 mode를 'full'로 지정. 즉, mode는 패딩의 유형을 뜻한다.
correlate(x, w, mode='full') # 배열의 원소 갯수가 10개임. x의 모든 원소들이 다 같은 횟수만큼 교차 상관 연산에 기여했다. 

array([ 6, 34, 51, 48, 57, 24, 25, 16, 39, 29, 13, 10])

In [14]:
# 세임 패딩(Same padding) : 출력값이 원본 배열(위 경우엔 x)의 길이와 같아지도록 원본 배열에 제로 패딩을 추가한 것
correlate(x, w, mode ='same') # 입력 배열의 크기는 9였으며 앞에는 2개, 뒤에는 1개의 제로 패딩이 추가됨.
# 합성곱 신경망에서는 대부분 세임 패딩을 사용한다. 

array([34, 51, 48, 57, 24, 25, 16, 39, 29])

## 스트라이드
### 밸리드 패딩, 세임 패딩, 세임 패딩은 스트라이드가 1임. 즉, 교차 상관 연산을 수행한 다음 오른쪽으로 한 칸만 이동한다. 
### 스트라이드를 2로 지정하면 연산을 수행한 다음 오른쪽으로 두 칸만 이동한다. 
### 합성곱 신경망은 보통 스트라이드를 1로 설정한다. 그리고 대부분 2차원 배열에 대한 합성곱을 수행한다. 

## 2차원 배열에서의 합성곱
### 1차원에서 한거랑 비슷하게 수행됨. 
### 수행 방향은 왼쪽에서 오른쪽으로, 위에서 아래로 1칸씩 이동하며 배열 원소끼리 곱하면 된다.
### 3*3 배열과 2*2배열의 합성곱을 수행할 때 2*2배열의 이동 방향이 z모양이다. 
### 글쓰는 것과 같은 방향이라고 생각하자

In [15]:
# 눈으로 확인해보기
x = np.array([[1,2,3], [4,5,6], [7,8,9]])
w = np.array([[2,0], [0,0]])
from scipy.signal import correlate2d
correlate2d(x, w, mode='valid') # valid 패딩으로 합성곱 연산 수행. valid 패딩으로 연산할 것이기에 크기가 줄어듦

array([[ 2,  4],
       [ 8, 10]])

In [17]:
# same 패딩을 사용함으로써 원본크기(3*3)와 같은 출력 배열(3*3)이 만들어짐
correlate2d(x, w, mode='same') 

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18]])

## 텐서플로우로 합성곱을 수행
### conv2d()로 2차원 합성곱을 수행. conv2d()는 입력값으로 4차원 배열을 받는다. 왜냐하면 입력 이미지의 높이, 너비 말고 다른 정보도 필요하기 때문이다. 바로 컬러값(R, G, B)다.
### 높이, 너비 크기의 2차원 배열이 만들어지고 배열의 원소값에 해당 원소 자리(픽셀)의 R, G, B 값이 들어간다. 2차원 배열이 3개 있어서 4차원 배열이다. (R값만 담긴 2차원 배열, G값만 담긴 2차원 배열, B값만 담긴 2차원 배열)
### w, x 모두 4차원 배열이다. x는 (배치, 샘플 높이, 샘플 너비, 컬러 채널의 차원)의 4차원이고 w는 (가중치의 높이, 가중치의 너비, 채널, 가중치의 개수)의 4차원이다. (일반적으로 합성곱의 입력과 가중치의 채널 갯수는 같다.)
### 입력 이미지의 배열은 (2,3,3,3)으로 표현하며 가중치는 (2,2,3,3)으로 나타낼 수 있다. 이해가 안된다면 책을 보자. 아무튼 이 두개를 합성곱 하면 (2,3,3,3) * (2,2,3,3) = (2,3,3,3)이 나온다. 
### 출력값은 (입력의 배치 ,입력의 높이, 입력의 너비, 필터의 개수)의 4차원 배열이다. 

In [18]:
# 2차원 배열을 4차원 배열로 바꿔 합성곱 수행
# 입력 x와 가중치 w를 넘파이의 reshape()로 2차원 배열에서 4차원 배열로 바꾸고 넘파이의 astype()으로 입력의 자료형을 실수로 바꿈. 왜냐하면 텐서플로우는 실수형의 입력값을 기대하기 때문. 

import tensorflow as tf
# 앞서 우리가 직관적으로 알 수 있던 2차원 배열을 하나로 압축한 거로 이해하면 되겠구나. 2차원 배열에 추가로 정보를 넣어 4차원 배열을 만든다. ㅎㄷㄷ
x_4d = x.astype(np.float).reshape(1,3,3,1) # 자료형을 실수로 바꿈. 배치와 컬러 채널은 1이다.
w_4d = w.reshape(2,2,1,1)

In [19]:
c_out = tf.nn.conv2d(x_4d, w_4d, strides=1, padding='SAME') # same 패딩으로 합성곱 연산 수행. conv2d()의 패딩 옵션은 대문자를 사용한다. 

In [21]:
# conv2d()는 출력값으로 Tensor객체를 반환. 텐서플로우에서는 다차원 배열을 텐서(tensor)라고 부른다. Tensor 객체의 numpy()를 사용하면 텐서를 넘파이 배열로 변환할 수 있음.
c_out.numpy().reshape(3,3) # 배치 차원과 컬러 차원을 제거하고 편의상 (3,3) 크기로 변환

array([[ 2.,  4.,  6.],
       [ 8., 10., 12.],
       [14., 16., 18.]])

### 앞서 옷 종류 판별(MNIST)할 때 계산을 위해 28*28을 784개의 1차원 배열로 만들었다. 
### 이렇게 되면 가중치도 많이 필요하며 그 결과 많은 연산을 수행해야만 한다. 
### 허나 합성곱 신경망에선 28*28을 784개로 펼치지 않고 그대로 사용하여 3*3 또는 5*5의 가중치로 합성곱을 적용한다. 
### w의 크기가 훨씬 작아졌고(784 * 100 = 74,800개에서 엄청 줄어듦) 입력의 특징을 더 잘찾기 때문에(왤까? 아직 이해가 안된다) 합성곱 신경망이 이미지 분류에서 뛰어난 성능을 발휘할 수 있다고 한다. 
### 전에 데이콘 수업 때 합성곱 연산이 컴퓨터에게 있어 더 적은 연산을 하게 되어 빠른 연산이 가능하게 한다고 배운게 기억난다. 
### 근데 이걸 실제 모델 디자인에 쓸 수 있다는게 아직 이해가 안된다. 더 많은 공부가 필요하다. 

### 텐서플로우의 케라스에서는 합성곱의 가중치를 커널이라고 부른다. 